In [ ]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import json

In [ ]:
def get_team_urls(standing_url):
    data = requests.get(standing_url)
    soup = bts(data.text)
    standing_table = soup.select("table.stats_table")[0]
    links = standing_table.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    return team_urls, soup.select("a.prev")[0].get("href")

def get_shooting_data(team_url):
    data = requests.get(team_url)
    soup = bts(data.text)
    links = soup.find_all("a")
    links = [l.get("href") for l in links]
    links = [l for l in links if l and "all_comps/shooting/" in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    try:
        shooting_data = pd.read_html(data.text, match="Shooting")[0]
        shooting_data.columns = shooting_data.columns.droplevel()
    except ValueError:
        shooting_data = None
    return shooting_data

def clean_team_name(team_url):
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
    return team_name

def fetch_data(year, standing_url):
    all_matches = []
    while standing_url is not None:
        team_urls, standing_url = get_team_urls(standing_url)
        for team_url in team_urls:
            team_name = clean_team_name(team_url)
            data = requests.get(team_url)
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
            shooting_data = get_shooting_data(team_url)
            if shooting_data is None:
                continue
            try:
                team_data = matches.merge(shooting_data[["Date", "Sh", "SoT", "Dist","FK", "PK", "PKatt"]], on="Date")
            except ValueError:
                continue
            team_data = team_data[team_data["Comp"] == "Premier League"]
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(1)
    return pd.concat(all_matches)
years = range(2021, 2022)
all_data = []
for year in years:
    standing_url = f"https://fbref.com/en/comps/9/{year}-Premier-League-Stats"
    all_data.append(fetch_data(year, standing_url))